# DB transformer

First set establish a postgres and install the dvdschema to it. This can be done using the following commands:

docker run --name some-postgres --rm -p 5432:5432 -e POSTGRES_HOST_AUTH_METHOD=trust -d postgres:12.15
    
docker exec some-postgres apt update -y

docker exec some-postgres apt install wget unzip -y

docker exec some-postgres wget https://www.postgresqltutorial.com/wp-content/uploads/2019/05/dvdrental.zip

docker exec some-postgres unzip dvdrental.zip

docker exec some-postgres psql -U postgres -c "CREATE DATABASE dvdrental;"

docker exec -i some-postgres pg_restore -U postgres -d dvdrental dvdrental.tar

docker exec some-postgres psql -U postgres -d dvdrental -c "create schema target;"


In [ ]:
!apt update -y -q
!apt install build-essential libpq-dev graphviz graphviz-dev -y
!pip install psycopg2
!pip install eralchemy2
!pip install graphviz
!pip install openai

In [ ]:
from eralchemy2 import render_er

## Draw from Postgres database
render_er("postgresql+psycopg2://postgres@0.0.0.0:5432/dvdrental", 'erd_from_postgres.png')


In [ ]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

img = np.asarray(Image.open('erd_from_postgres.png'))
plt.rcParams["figure.figsize"] = (15, 10)
plt.imshow(img)
plt.show()


In [ ]:
conn.close()

In [ ]:
import psycopg2
conn = psycopg2.connect("host=0.0.0.0 dbname=dvdrental user=postgres")
cur = conn.cursor()


In [ ]:
from flask import request
import os
import openai
openai.api_key = os.getenv('YOUR_ENV_VAR')

In [ ]:
def createweirdtarget01(targetschema='target',constraint=True):
    sql = "DROP SCHEMA IF EXISTS "+targetschema+" CASCADE;"
    try:
        cur.execute(sql)
    except Exception as err:
        print(err)
        er=err
    conn.commit()

    sql = "CREATE SCHEMA "+targetschema+";"
    try:
        cur.execute(sql)
    except Exception as err:
        print(err)
        er=err
    conn.commit()

    sql="CREATE TABLE target.movie (\
    m_id SERIAL PRIMARY KEY,\
    title TEXT NOT NULL,\
    category TEXT NOT NULL,\
    rental_date TIMESTAMP NOT NULL\
);"
    cur.execute(sql)
    if constraint:
        sql="CREATE TABLE target.info (\
            email TEXT NOT NULL,\
            address TEXT NOT NULL,\
            m_id INTEGER NOT NULL,\
            FOREIGN KEY (m_id) REFERENCES target.movie(m_id)\
        );"
    else:
        sql="CREATE TABLE target.info (\
            email TEXT NOT NULL,\
            address TEXT NOT NULL,\
            m_id INTEGER NOT NULL\
        );"        
    cur.execute(sql)
    conn.commit()
    sql="select a.address,b.email, e.title, c.rental_date, g.name from \
    public.address a, \
    public.customer b, \
    public.rental c, \
    public.inventory d, \
    public.film e, \
    public.film_category f, \
    public.category g \
    where \
    a.address_id=b.address_id and \
    b.customer_id=c.customer_id and \
    c.inventory_id=d.inventory_id and \
    d.film_id=e.film_id and \
    e.film_id=f.film_id and \
    f.category_id=g.category_id \
    order by name desc limit 10"
    cur.execute(sql)
    a=cur.fetchall()
    for i in range(10):
        sql="INSERT INTO target.movie (title, category, rental_date,m_id) VALUES ('"+a[i][2]+"','"+a[i][4]+"','"+a[i][3].strftime("%Y-%m-%d")+"','"+str(i)+"');"
        cur.execute(sql)
        conn.commit()
        sql="INSERT INTO target.info (email,address,m_id) VALUES ('"+a[i][1]+"','"+a[i][0]+"','"+str(i)+"');"
        cur.execute(sql)
        conn.commit()
        
        


In [ ]:
def createweirdtarget02(targetschema='target',constraint=True,insertdata=True):
    sql = "DROP SCHEMA IF EXISTS "+targetschema+" CASCADE;"
    try:
        cur.execute(sql)
    except Exception as err:
        print(err)
        er=err
    conn.commit()

    sql = "CREATE SCHEMA "+targetschema+";"
    try:
        cur.execute(sql)
    except Exception as err:
        print(err)
        er=err
    conn.commit()

    sql="CREATE TABLE target.movie (\
    m_id SERIAL PRIMARY KEY,\
    title TEXT NOT NULL,\
    category TEXT NOT NULL,\
    language TEXT,\
    rental_date TIMESTAMP NOT NULL\
);"
    cur.execute(sql)
    if constraint:
        sql="CREATE TABLE target.info (\
            email TEXT NOT NULL,\
            address TEXT NOT NULL,\
            m_id INTEGER NOT NULL,\
            FOREIGN KEY (m_id) REFERENCES target.movie(m_id)\
        );"
    else:
        sql="CREATE TABLE target.info (\
            email TEXT NOT NULL,\
            address TEXT NOT NULL,\
            m_id INTEGER NOT NULL\
        );"        
    cur.execute(sql)
    conn.commit()
    sql="select a.address,b.email, e.title, c.rental_date, g.name, h.name as language from \
    public.address a, \
    public.customer b, \
    public.rental c, \
    public.inventory d, \
    public.film e, \
    public.film_category f, \
    public.category g, \
    public.language h \
    where \
    a.address_id=b.address_id and \
    b.customer_id=c.customer_id and \
    c.inventory_id=d.inventory_id and \
    d.film_id=e.film_id and \
    e.film_id=f.film_id and \
    f.category_id=g.category_id and \
    e.language_id=h.language_id \
    order by name desc limit 10"
    cur.execute(sql)
    a=cur.fetchall()
    if insertdata:
        for i in range(10):
            sql="INSERT INTO target.movie (title, category, rental_date,language, m_id) VALUES ('"+a[i][2]+"','"+a[i][4]+"','"+a[i][3].strftime("%Y-%m-%d")+"','"+a[i][5]+"','"+str(i)+"');"
            cur.execute(sql)
            conn.commit()
            sql="INSERT INTO target.info (email,address,m_id) VALUES ('"+a[i][1]+"','"+a[i][0]+"','"+str(i)+"');"
            cur.execute(sql)
            conn.commit()
        
        


In [ ]:
createweirdtarget02(constraint=False)

In [ ]:
import os
def givetabledef(schema,table):
    com="pg_dump -U postgres --schema-only -h 0.0.0.0 dvdrental -t '"+schema+"."+table+"' > dump.sql"
    os.system(com)
    flag=False
    with open('dump.sql', 'r') as f:
        lines = f.readlines()
    result=[]
    for line in lines:
        if ('CREATE' in line) or ('ALTER' in line):
            flag=True
        if flag:
            result.append(line)
        if flag and ';' in line:
            flag=False
    return result

In [ ]:
def givetableconstraints(schema,table):
    com="pg_dump -U postgres --schema-only -h 0.0.0.0 dvdrental -t '"+schema+"."+table+"' > dump.sql"
    os.system(com)
    flag=False
    with open('dump.sql', 'r') as f:
        lines = f.readlines()
    result=[]
    for line in lines:
        if  ('ALTER' in line):
            flag=True
        if flag:
            result.append(line)
        if flag and ';' in line:
            flag=False
    return result

In [ ]:
givetableconstraints('public','address')

In [ ]:
import random
import string

def generate_random_string(length):
    letters = string.ascii_letters
    return (''.join(random.choice(letters) for i in range(length))).lower()

random_string = generate_random_string(10)
print(random_string)


In [ ]:
def getexampledata(schema,table,column="*"):
    sql="select "+column+" from "+schema+"."+table+" limit 10";
    #print(sql)
    cur.execute(sql)
    res=cur.fetchall()
    #print(res)
    results=[]
    for r in res:
        results.append(str(r[1:]))
    return(results)


In [ ]:
getexampledata('public','address')

In [ ]:
def ddlemb(schema):
    #schema='public'
    sql="SELECT table_name FROM information_schema.tables WHERE table_schema = '"+schema+"' ORDER BY table_name;"
    cur.execute(sql)
    res=cur.fetchall()


    MODEL = "text-embedding-ada-002"

    #res
    #cosine = np.dot(A,B)/(norm(A)*norm(B))

    emb={}
    for a in res:
        #s=givetabledef(schema,a[0])
        s=givetableconstraints(schema,a[0])
        r = openai.Embedding.create(input=[''.join(s)], engine=MODEL)
        emb[a[0]]=r['data'][0]['embedding']
    return emb
    


sql="SELECT con.* \
FROM pg_catalog.pg_constraint con \
INNER JOIN pg_catalog.pg_class rel \
ON rel.oid = con.conrelid \
INNER JOIN pg_catalog.pg_namespace nsp \
ON nsp.oid = connamespace \
WHERE nsp.nspname = 'public' \
AND rel.relname = 'address';"

cur.execute(sql)
a=cur.fetchall()
a


In [ ]:
givetabledef('public','address')

In [ ]:
def columnemb_void(schema):
    emb={}
    MODEL = "text-embedding-ada-002"

    sql="SELECT table_name FROM information_schema.tables WHERE table_schema = '"+schema+"' ORDER BY table_name;"
    cur.execute(sql)
    rtable=cur.fetchall()

    for res_table in rtable:
        table=res_table[0]
        sql="SELECT column_name as cn FROM information_schema.columns WHERE table_schema = '"+schema+"' AND table_name = '"+table+"';"
        cur.execute(sql)
        res=cur.fetchall()
        for r in res:
            column=r[0]
            sql="select '"+r[0]+"' from "+schema+"."+table+" limit 10";
            cur.execute(sql)
            res1=cur.fetchall()
            s=""
            for rr in res1:
                s=s+str(rr[0])+"\n" 
            rrr = openai.Embedding.create(input=[''.join(s)], engine=MODEL)
            v=rrr['data'][0]['embedding']
            #v=89
            k={"schema":schema,"table":table,"column":column,"examples":s,"emb":v}
            key=schema+"|"+table+"|"+column.replace(" ","-")
            emb[key]=v
    return emb


In [ ]:
import pandas as pd
def columnexampleembmatrix(schema):
    emb={}
    MODEL = "text-embedding-ada-002"

    sql="SELECT table_name FROM information_schema.tables WHERE table_schema = '"+schema+"' ORDER BY table_name;"
    cur.execute(sql)
    rtable=cur.fetchall()

    for res_table in rtable:
        table=res_table[0]
        sql="SELECT column_name as cn FROM information_schema.columns WHERE table_schema = '"+schema+"' AND table_name = '"+table+"';"
        cur.execute(sql)
        res=cur.fetchall()
        for r in res:
            column=r[0]
            #print(table + '|' + column)
            sql='select "'+r[0]+'" from '+schema+'.'+table+' limit 10;'
            #print(sql)
            cur.execute(sql)
            res1=cur.fetchall()
            w=[]
            for rr in res1:
                s=str(rr[0])
                #print(s)
                if (s is not None):
                    if s != '':
                        rrr = openai.Embedding.create(input=[s], engine=MODEL)
                        v=rrr['data'][0]['embedding']
                        w.append(v)
            if len(w)==10:
                keyname=schema+'|'+table+'|'+column
                emb[keyname]=pd.DataFrame(w)
    return emb

In [ ]:
mtarget=columnexampleembmatrix('target')

In [ ]:
mtarget

In [ ]:
mpublic=columnexampleembmatrix('public')

In [ ]:
def columncor(column_emb_source,column_emb_target):
    results=[]
    examples={}
    for k in column_emb_target.keys():
        targetschema=k.split("|")[0]
        targettable=k.split("|")[1]
        targetcolumn=k.split("|")[2]
        targetcolumn='"'+k.split("|")[2]+'"'
        
        sql='select '+targetcolumn +' from '+targetschema+'.'+targettable+' limit 10;'
        cur.execute(sql)
        res=cur.fetchall()
        conn.commit()
        examples[k]=[r[0] for r in res]

        #res=[]
        targetname=[]
        cor=[]
        for l in column_emb_source.keys():
            a=pd.DataFrame(column_emb_source[l])
            b=pd.DataFrame(column_emb_target[k])
            #print(k)
            #print(l)
            V=a.dot(b.transpose())
            v=V.mean().mean()
            targetname.append(l)
            sourceschema=l.split("|")[0]
            sourcetable=l.split("|")[1]
            #sourcecolumn="'"+l.split("|")[2]+"'"
            sourcecolumn='"'+l.split("|")[2]+'"'
            #sourceex=getexampledata(sourceschema,sourcetable,sourcecolumn)
            sql='select '+sourcecolumn +' from '+sourceschema+'.'+sourcetable+' limit 10;'
            cur.execute(sql)
            res=cur.fetchall()
            conn.commit()
            cor.append(v)
            examples[l]=[r[0] for r in res]
        df=pd.DataFrame({'target':targetname,'cor':cor})
        df=df.sort_values(by='cor',ascending=False)
        a={'target':k,'source':list(df['target'])}
        results.append(a)
    return results, [examples]

In [ ]:
res,examples=columncor(mpublic,mtarget)

In [ ]:
res

In [ ]:
def printcor(results,examples):
    for r in results:
        k=r['target']
        for l in r['source']:
            print("Target: {}, with examples {} \n has potential source {}\n with examples {}\n\n".format(k,examples[0][k][:3],l,examples[0][l][:3]))

In [ ]:
printcor(res,examples)

In [ ]:
columnmaps={}
columnmaps['target.info.email']='public.customer.email'
columnmaps['target.info.address']='public.address.address'
#columnmaps['target|movie|title']='public|film|title'#skipping this one, very low on the list
columnmaps['target.movie.category']='public.category.name'
columnmaps['target.movie.rental_date']='public.rental.rental_date'#should also skip??


In [ ]:
import re

class Prompt():
    def __init__(self,sourceschema,targetschema,targettable,columnmappings):
        self.sourcetables=[]
        self.sourceschema=sourceschema
        self.targetschema=targetschema
        self.targettable=targettable
        self.columnmappings=columnmappings
        self.messages=[]
        self.ddlemb=ddlemb(self.sourceschema)


    def initialmessage(self):
        prompt={
          "role": "system",
          "content": "Generate sql code in a separate message."
        }
        self.messages.append(prompt)

        prompt={
          "role": "user",
          "content": "You are a developer writing SQL queries. Given the following SQL tables and example data from them, your job is to write querie(s) that moves data from the schema"+self.sourceschema+ "to the "+self.targetschema+"."+self.targettable+" table. You might need to join tables in order to do so."     
        }
        self.messages.append(prompt)
        
        prompt={
        "role":"user",
        "content":"Table "+self.targetschema+"."+self.targettable+" has DDL:"
        }
        self.messages.append(prompt)
        s=givetabledef(self.targetschema,self.targettable)
        prompt={
        "role":"user",
        "content":''.join(s)
        }
        self.messages.append(prompt)            

        prompt={
          "role": "user",
          "content": "Also provide sql code to take data from the table "+self.targetschema+"."+self.targettable+" and put it back at its origin in the schema "+self.sourceschema
        }
        #self.messages.append(prompt)

        prompt={
          "role": "user",
          "content": "Please explain your reasoning."
        }
        self.messages.append(prompt)


    def addmappings(self):
        prompt={
          "role": "user",
          "content": "Mappings between some of the "+self.sourceschema+" columns and columns in "+self.targetschema+"."+self.targettable+" are:"
        } 
        for k in self.columnmappings.keys():
            targetschema=k.split(".")[0]
            targettable=k.split(".")[1]
            targetcolumn=k.split(".")[2]
            if targetschema==self.targetschema:
                if targettable==self.targettable:
                    prompt={
                      "role": "user",
                      "content": k+":"+self.columnmappings[k]
                    } 
                    self.messages.append(prompt)

    def addddl(self):
        for tab in self.sourcetables:
            prompt={
            "role":"user",
            "content":"Table "+self.sourceschema+"."+tab+" has DDL:"
            }
            self.messages.append(prompt)
            s=givetabledef(self.sourceschema,tab)
            prompt={
            "role":"user",
            "content":''.join(s)
            }
            self.messages.append(prompt)

            prompt={
            "role":"user",
            "content":"Some example data from this table is the following:"
            }
            #self.messages.append(prompt)
            s=getexampledata(self.sourceschema,tab,"*")
            for ss in s[:2]:
                prompt={"role":"user",
                       "content":ss}
                #self.messages.append(prompt)
                
    #def givetabledef(schema,table):

    def addsourcetables(self):
        if len(self.sourcetables)>0:
            for _ in range(2):
                B=pd.DataFrame(self.ddlemb)
                A=B[self.sourcetables]
                C=B[[k for k in self.ddlemb.keys() if k not in self.sourcetables]]
                V=C.transpose().dot(A)
                max_col = V.max().idxmax()
                max_row = V[max_col].idxmax()
                max_row = V.sum(axis=1).idxmax()
                self.sourcetables.append(max_row)
        if len(self.sourcetables)==0:
            for k in self.columnmappings.keys():
                #self.alreadythere=[self.ddlemb[k] ]
                l=self.columnmappings[k]
                sourceschema=l.split(".")[0]
                sourcetable=l.split(".")[1]
                sourcecolumn=l.split(".")[2]
                targetschema=k.split(".")[0]
                targettable=k.split(".")[1]
                targetcolumn=k.split(".")[2]
                if targettable==self.targettable:
                    self.sourcetables.append(sourcetable)
                    
    def getsql(self):
        r=openai.ChatCompletion.create(
          model="gpt-3.5-turbo-16k",
          #model="gpt-4",
          messages=self.messages
        )
        #print(self.messages)
        #print(r)
        text=r['choices'][0]['message']['content']
        match = re.search(r"sql\n([\s\S]*?)```\n", text) 
        sql = ""
        if match: 
            sql = match.group(1) 
        return(sql)
    
    def testsql(self,sql):
        createweirdtarget02(constraint=False,insertdata=False)
        try:
            cur.execute(sql)
        except Exception as err:
            conn.rollback()
            #print(err)
            er=str(err)
            return er
        conn.commit()
        return ""
    
    def buildsql(self):
        #print(self.messages)
        for j in range(8):
            self.messages=[]
            self.initialmessage()
            self.addmappings()
            self.addsourcetables()
            self.addddl()
            #print(len(self.messages))
            print(self.sourcetables)
            for i in range(2):
                sql=self.getsql()
                er=self.testsql(sql)
                #conn.close()
                #conn = psycopg2.connect("host=0.0.0.0 dbname=dvdrental user=postgres")
                #cur = conn.cursor()
                if er!="":
                    #try again, feedback error
                    prompt={"role":"user",
                   "content":"You answered with this sql:"}
                    self.messages.append(prompt)
                    prompt={"role":"user",
                   "content":sql}
                    self.messages.append(prompt)
                    print(sql)
                    print(er)
                    prompt={"role":"user",
                   "content":"Which unfortunately gave this error:"+er}
                    self.messages.append(prompt)
                    prompt={"role":"user",
                   "content":"Can you try again?"}
                    self.messages.append(prompt)
                else:
                    print("succes")
                    return sql
        return None
    


In [ ]:
createweirdtarget02(constraint=False)

In [ ]:
p=Prompt(sourceschema='public',targetschema='target',targettable='info',columnmappings=columnmaps)

In [ ]:
r=p.buildsql()

In [ ]:
from pprint import pprint
pprint(r)

In [ ]:
createweirdtarget02(constraint=False)

In [ ]:
p=Prompt(sourceschema='public',targetschema='target',targettable='movie',columnmappings=columnmaps)

In [ ]:
r=p.buildsql()

In [ ]:
from pprint import pprint
pprint(r)

In [ ]:
from pprint import pprint
pprint(r)

In [ ]:
from pprint import pprint
pprint(r)

In [ ]:
p.messages

In [ ]:
r=openai.ChatCompletion.create(
  model="gpt-3.5-turbo-16k",
  #model="gpt-4",
  messages=p.messages
)
r


In [ ]:
text=r['choices'][0]['message']['content']
match = re.search(r"sql\n([\s\S]*?)```\n", text) 
sql = ""
if match: 
    sql = match.group(1) 


In [ ]:
sql

In [ ]:
pprint(sql)

In [ ]:
p.testsql(sql)